<a href="https://colab.research.google.com/github/gxilly/bsc_dpdm22/blob/main/%5Bdataprep1%5D_property_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os
os.chdir('/content/gdrive/MyDrive/bscdata')

!pwd #เช็ค paht ที่กำลังทำงานอยู่

/content/gdrive/MyDrive/bscdata


- ดูจากค่าอื่นๆ ที่ดินนี้อยู่ที่ไหน พื้นที่เท่าไหร่ ตร.ละเท่าไหร่ feature ต่างๆ เอาค่าพวกนี้มาสร้าง model ก่อน แล้วเอาค่าที่มีอยู่ มาทำนายค่าที่หายไป ## ดีเยี่ยม!!
- 0 เกิน 0 ขาด ส่วนใหญ่น่าจะตกแค่ตัวเดียว ต่างกัน 10 เท่า
ควรเอาราคาสองอันมาหารกัน (ประเมิน/ตั้งขาย, ตั้งขาย/ประเมิน) ห่างกัน 9-10,0.1 

# import ข้อมูล

## House

In [5]:
# open file
house = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=0)
print(house.shape) # shown number of (row, column)
house.head()

(190, 14)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1,https://nayoo.co/khonkaen/posts/25,บ้านเดี่ยว,มั่นใจ,2700000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,100.0
1,2,https://nayoo.co/khonkaen/posts/25,บ้านเดี่ยว,มั่นใจ,2700000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,NaN,NaN,NaN,NaN,NaN
2,1,https://nayoo.co/khonkaen/posts/33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN,NaN
3,2,https://nayoo.co/khonkaen/posts/33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN
4,1,https://nayoo.co/khonkaen/posts/495,บ้านเดี่ยว,มั่นใจ,5900000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN


## Land (ที่ดิน)

In [6]:
land = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=1)
print(land.shape) 
land.head()

(210, 13)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,https://nayoo.co/khonkaen/posts/1033,ที่ดิน,มั่นใจ,2800000,1800000.0,"8,000-25,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3.0,ขายได้ ใช้ระยะเวลา 7-12 เดือน,NaN,100.0
1,2,https://nayoo.co/khonkaen/posts/1033,ที่ดิน,มั่นใจ,2800000,2000000.0,"ราคาเสนอขาย 7,000-12,0000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN
2,4,https://nayoo.co/khonkaen/posts/1033,ที่ดิน,มั่นใจ,2800000,3000000.0,12000-18000,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3.0,ขายได้ ใช้ระยะเวลา 7-12 เดือน,NaN,NaN
3,1,https://nayoo.co/khonkaen/posts/1711,ที่ดิน,มั่นใจ,14900000,8000000.0,"25,000-80,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN
4,2,https://nayoo.co/khonkaen/posts/1711,ที่ดิน,มั่นใจ,14900000,12000000.0,"ราคาเสนอขาย 100,000-150,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN


## Condo

In [7]:
condo = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=2)
print(condo.shape)
condo.head()

(10, 11)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10
0,1,32.0,คอนโด,มั่นใจ,3500000.0,3500000.0,"60,000-130,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1,1,36.0,คอนโด,มั่นใจ,1930000.0,2200000.0,"50,000-90,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
2,1,603.0,คอนโด,มั่นใจ,2438000.0,2800000.0,"35,000-80,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,1,1395.0,คอนโด,มั่นใจ,1600000.0,2300000.0,"30,000-80,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,1,1443.0,คอนโด,มั่นใจ,1860000.0,1800000.0,"30,000-80,000 บาท/ตารางเมตร",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี


## Town home

In [8]:
tw = pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=3)
print(tw.shape) 
tw.head()

(240, 13)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,https://nayoo.co/khonkaen/posts/548,ทาวน์โฮม,มั่นใจ,1045000.0,700000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,120.0
1,2,https://nayoo.co/khonkaen/posts/548,ทาวน์โฮม,มั่นใจ,1045000.0,1500000.0,ราคาซื้อขาย 1-2 ล้าน/คูหา,NaN,1 ขายไม่ได้แน่นอน,NaN,NaN,NaN,NaN
2,1,https://nayoo.co/khonkaen/posts/549,ทาวน์โฮม,มั่นใจ,935000.0,700000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,NaN
3,2,https://nayoo.co/khonkaen/posts/549,ทาวน์โฮม,มั่นใจ,935000.0,1500000.0,ราคาเสนอขาย 1-2 ล้าน/คูหา,NaN,1 ขายไม่ได้แน่นอน,NaN,NaN,NaN,NaN
4,1,https://nayoo.co/khonkaen/posts/579,ทาวน์โฮม,มั่นใจ,3520000.0,3500000.0,"8,000-80,000",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,NaN,NaN,NaN,NaN


## Shop house (อาคารพาณิชย์)

In [9]:
sh= pd.read_excel('คำตอบนักประเมิน.xlsx',sheet_name=4)
print(sh.shape) 
sh.head()

(20, 14)


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1,https://nayoo.co/khonkaen/posts/6,อาคารพาณิชย์/สำนักงาน,มั่นใจ,7000000.0,6000000.0,"8,000-30,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3.0,ขายได้ ใช้ระยะเวลา 7-12 เดือน,NaN,NaN,10.0
1,2,https://nayoo.co/khonkaen/posts/6,อาคารพาณิชย์/สำนักงาน,มั่นใจ,7000000.0,8000000.0,"ราคาเสนอขาย 4,000,000-6,000,000 บาท/คูหา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN
2,1,https://nayoo.co/khonkaen/posts/17,อาคารพาณิชย์/สำนักงาน,มั่นใจ,12000000.0,12000000.0,"8,000-40,000",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN
3,2,https://nayoo.co/khonkaen/posts/17,อาคารพาณิชย์/สำนักงาน,มั่นใจ,12000000.0,12000000.0,ราคาเสนอขาย 4 - 5 ล้านบาท/คูหา,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,NaN
4,1,https://nayoo.co/khonkaen/posts/21,อาคารพาณิชย์/สำนักงาน,มั่นใจ,22500000.0,13500000.0,"8,000-50,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,NaN


# สรุปข้อมูลของนักประเมินแต่ละคน โดยพิจารณาจาก ความมั่นใจในการประเมิน ประเมินราคาซื้อ-ขาย ให้คะแนนทรัพย์

## House

In [10]:
house['ID ผู้ประเมิน'].value_counts()

1    100
2     90
Name: ID ผู้ประเมิน, dtype: int64

In [11]:
est1 = house[house['ID ผู้ประเมิน']==1]

In [12]:
data01 = est1[['ID ผู้ประเมิน','ความมั่นใจในการเมินราคาทรัพย์','ประเมินราคาซื้อ-ขาย','ให้คะแนนทรัพย์']]
data01.head(5)

,ID ผู้ประเมิน,ความมั่นใจในการเมินราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ให้คะแนนทรัพย์
0,1,มั่นใจ,2500000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
2,1,มั่นใจ,2200000.0,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,1,มั่นใจ,5000000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
6,1,มั่นใจ,3500000.0,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
8,1,มั่นใจ,2500000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [13]:
est1.describe()

,ID ผู้ประเมิน,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
count,100.0,1.000000e+02,1.000000e+02,0.0,0.0,0.0,0.0,1.0
mean,1.0,4.316360e+06,3.938500e+06,NaN,NaN,NaN,NaN,100.0
std,0.0,2.865391e+06,2.368553e+06,NaN,NaN,NaN,NaN,NaN
min,1.0,3.900000e+05,6.000000e+05,NaN,NaN,NaN,NaN,100.0
25%,1.0,2.892750e+06,2.775000e+06,NaN,NaN,NaN,NaN,100.0
50%,1.0,3.962000e+06,3.600000e+06,NaN,NaN,NaN,NaN,100.0
75%,1.0,5.000000e+06,4.500000e+06,NaN,NaN,NaN,NaN,100.0
max,1.0,2.094900e+07,1.600000e+07,NaN,NaN,NaN,NaN,100.0


In [14]:
est1['ให้คะแนนทรัพย์'].value_counts()

2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี    54
3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน     43
4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน       2
1 ขายไม่ได้แน่นอน                      1
Name: ให้คะแนนทรัพย์, dtype: int64

In [15]:
est1['ความมั่นใจในการเมินราคาทรัพย์'].value_counts()

มั่นใจ    100
Name: ความมั่นใจในการเมินราคาทรัพย์, dtype: int64

In [16]:
est2 = house[house['ID ผู้ประเมิน']==2]

In [17]:
data02 =est2[['ID ผู้ประเมิน','ความมั่นใจในการเมินราคาทรัพย์','ประเมินราคาซื้อ-ขาย','ให้คะแนนทรัพย์']]
data02.head(5)

,ID ผู้ประเมิน,ความมั่นใจในการเมินราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ให้คะแนนทรัพย์
1,2,มั่นใจ,2700000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
3,2,มั่นใจ,2400000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
5,2,มั่นใจ,9000000.0,5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน
7,2,มั่นใจ,3700000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
9,2,มั่นใจ,2500000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [18]:
est2.describe()

,ID ผู้ประเมิน,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
count,90.0,9.000000e+01,8.900000e+01,0.0,0.0,0.0,0.0,0.0
mean,2.0,4.240733e+06,4.153258e+06,NaN,NaN,NaN,NaN,NaN
std,0.0,2.714815e+06,2.386901e+06,NaN,NaN,NaN,NaN,NaN
min,2.0,3.900000e+05,9.900000e+05,NaN,NaN,NaN,NaN,NaN
25%,2.0,2.860000e+06,2.600000e+06,NaN,NaN,NaN,NaN,NaN
50%,2.0,4.000000e+06,3.900000e+06,NaN,NaN,NaN,NaN,NaN
75%,2.0,5.000000e+06,4.900000e+06,NaN,NaN,NaN,NaN,NaN
max,2.0,2.094900e+07,1.700000e+07,NaN,NaN,NaN,NaN,NaN


In [19]:
est2['ให้คะแนนทรัพย์'].value_counts()

3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน     64
4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน      23
5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน      1
2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี     1
Name: ให้คะแนนทรัพย์, dtype: int64

In [20]:
est2['ความมั่นใจในการเมินราคาทรัพย์'].value_counts()

มั่นใจ       89
ไม่มั่นใจ     1
Name: ความมั่นใจในการเมินราคาทรัพย์, dtype: int64

## Land

In [21]:
land['ID ผู้ประเมิน'].value_counts() 

1    100
2    100
4     10
Name: ID ผู้ประเมิน, dtype: int64

ใช้แค่ ID 01 กับ 02

In [22]:
est_1l = land[land['ID ผู้ประเมิน']==1]

In [23]:
est_2l = land[land['ID ผู้ประเมิน']==2]

## Condo

In [24]:
condo['ID ผู้ประเมิน'].value_counts() 

1    10
Name: ID ผู้ประเมิน, dtype: int64

ใน condo มีผู้ประเมินคนเดียว คือ ID 01

## Town home

In [25]:
tw['ID ผู้ประเมิน'].value_counts() 

1    120
2    120
Name: ID ผู้ประเมิน, dtype: int64

In [26]:
est_1t = tw[tw['ID ผู้ประเมิน']==1]

In [27]:
est_2t =  tw[tw['ID ผู้ประเมิน']==2]

## Shop house

In [28]:
sh['ID ผู้ประเมิน'].value_counts() 

1    10
2    10
Name: ID ผู้ประเมิน, dtype: int64

In [29]:
est_1s = sh[sh['ID ผู้ประเมิน']==1]

In [30]:
est_2s =  sh[sh['ID ผู้ประเมิน']==2]

#เปรียบเทียบความแตกต่างในการประเมิณราคาทรัพย์แต่ละทรัพย์ของนักประเมินแต่ละคนและของราคาตั้งขาย

In [31]:
from sklearn.metrics import mean_absolute_percentage_error as mape #mape

## House

In [32]:
est2['ประเมินโดย 1'] = est2['post id'].map(est1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
est2.head(3)

<ipython-input-32-3501914803de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est2['ประเมินโดย 1'] = est2['post id'].map(est1.set_index('post id')['ประเมินราคาซื้อ-ขาย'])


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/25,บ้านเดี่ยว,มั่นใจ,2700000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,NaN,NaN,NaN,NaN,NaN,2500000.0
3,2,https://nayoo.co/khonkaen/posts/33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,NaN,2200000.0
5,2,https://nayoo.co/khonkaen/posts/495,บ้านเดี่ยว,มั่นใจ,5900000.0,9000000.0,"โซนนี้ราคาซื้อขาย ประมาณ 10,000-20,000 บาท/ตาร...",NaN,5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน,NaN,NaN,NaN,NaN,NaN,5000000.0


In [33]:
est2.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 2'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [34]:
ass_h = est2[['post id','ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2']]
ass_h

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2
1,https://nayoo.co/khonkaen/posts/25,2700000.0,2500000.0,2700000.0
3,https://nayoo.co/khonkaen/posts/33,2600000.0,2200000.0,2400000.0
5,https://nayoo.co/khonkaen/posts/495,5900000.0,5000000.0,9000000.0
7,https://nayoo.co/khonkaen/posts/564,4400000.0,3500000.0,3700000.0
9,https://nayoo.co/khonkaen/posts/566,2750000.0,2500000.0,2500000.0
...,...,...,...,...
185,https://nayoo.co/khonkaen/posts/19495,5500000.0,4500000.0,5000000.0
186,https://nayoo.co/khonkaen/posts/19496,7911000.0,5200000.0,7400000.0
187,https://nayoo.co/khonkaen/posts/20,2000000.0,3600000.0,2500000.0
188,https://nayoo.co/khonkaen/posts/31,4000000.0,4000000.0,3900000.0


เช็คและแก้ missing

In [35]:
ass_h.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2     True
dtype: bool

In [36]:
# ass_h = ass_h.fillna(value={'ประเมินโดย 2':est2['ประเมินโดย 2'].mean()}) #ควร drop

In [37]:
ass_h = ass_h.dropna(subset=['ประเมินโดย 2'])

ดูจากค่าอื่นๆ ที่ดินนี้อยู่ที่ไหน พื้นที่เท่าไหร่ ตร.ละเท่าไหร่ feature ต่างๆ เอาค่าพวกนี้มาสร้าง model ก่อน แล้วเอาค่าที่มีอยู่ มาทำนายค่าที่หายไป ## ดีเยี่ยม!!

In [38]:
ass_h.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2    False
dtype: bool

### MAPE

In [39]:
actual_h = ass_h[['ราคาทรัพย์']]
pred_1h = ass_h[['ประเมินโดย 1']]
pred_2h = ass_h[['ประเมินโดย 2']]

In [40]:
mph1 = mape(actual_h,pred_1h)
mph2 = mape(actual_h,pred_2h)
mph12 = mape(pred_1h,pred_2h)
mph21 = mape(pred_2h,pred_1h)

In [41]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mph1:.2f}\n MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ {mph2:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ {mph12:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ {mph21:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.12
 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ 0.10
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ 0.14
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ 0.12


### CHECK function ตรวจสอบค่าที่เติม 0 ผิด

In [87]:
def check_1h(in1,in2):
    return max(in1/in2,in2/in2)

Check1 = list()
for row in ass_h.iterrows() :
    print(row)

In [91]:
check1 = list()
for row in ass_h.iterrows() :
    check1.appen(lambda(check_1h(row[1]['ราคาทรัพย์'],row[1]['ประเมินโดย 1']))


SyntaxError: ignored

In [88]:
ass_h['Check1'] = ass_h.apply(lambda x:check_1h(x['ราคาทรัพย์'],x['ประเมินโดย 1']), axis = 1)
ass_h

<ipython-input-88-761c07b2d07e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_h['Check1'] = ass_h.apply(lambda x:check_1h(x['ราคาทรัพย์'],x['ประเมินโดย 1']), axis = 1)


,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1
1,https://nayoo.co/khonkaen/posts/25,2700000.0,2500000.0,2700000.0,1.080000
3,https://nayoo.co/khonkaen/posts/33,2600000.0,2200000.0,2400000.0,1.181818
5,https://nayoo.co/khonkaen/posts/495,5900000.0,5000000.0,9000000.0,1.180000
7,https://nayoo.co/khonkaen/posts/564,4400000.0,3500000.0,3700000.0,1.257143
9,https://nayoo.co/khonkaen/posts/566,2750000.0,2500000.0,2500000.0,1.100000
...,...,...,...,...,...
185,https://nayoo.co/khonkaen/posts/19495,5500000.0,4500000.0,5000000.0,1.222222
186,https://nayoo.co/khonkaen/posts/19496,7911000.0,5200000.0,7400000.0,1.521346
187,https://nayoo.co/khonkaen/posts/20,2000000.0,3600000.0,2500000.0,1.000000
188,https://nayoo.co/khonkaen/posts/31,4000000.0,4000000.0,3900000.0,1.000000


In [92]:
ass_h['Check2'] = ass_h.apply(lambda x:check_1h(x['ราคาทรัพย์'],x['ประเมินโดย 2']), axis = 1)
ass_h

<ipython-input-92-999252f84e15>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_h['Check2'] = ass_h.apply(lambda x:check_1h(x['ราคาทรัพย์'],x['ประเมินโดย 2']), axis = 1)


,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
1,https://nayoo.co/khonkaen/posts/25,2700000.0,2500000.0,2700000.0,1.080000,1.000000
3,https://nayoo.co/khonkaen/posts/33,2600000.0,2200000.0,2400000.0,1.181818,1.083333
5,https://nayoo.co/khonkaen/posts/495,5900000.0,5000000.0,9000000.0,1.180000,1.000000
7,https://nayoo.co/khonkaen/posts/564,4400000.0,3500000.0,3700000.0,1.257143,1.189189
9,https://nayoo.co/khonkaen/posts/566,2750000.0,2500000.0,2500000.0,1.100000,1.100000
...,...,...,...,...,...,...
185,https://nayoo.co/khonkaen/posts/19495,5500000.0,4500000.0,5000000.0,1.222222,1.100000
186,https://nayoo.co/khonkaen/posts/19496,7911000.0,5200000.0,7400000.0,1.521346,1.069054
187,https://nayoo.co/khonkaen/posts/20,2000000.0,3600000.0,2500000.0,1.000000,1.000000
188,https://nayoo.co/khonkaen/posts/31,4000000.0,4000000.0,3900000.0,1.000000,1.025641


In [95]:
ass_h.sort_values(by=['Check1'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
186,https://nayoo.co/khonkaen/posts/19496,7911000.0,5200000.0,7400000.0,1.521346,1.069054
111,https://nayoo.co/khonkaen/posts/6045,2915000.0,2000000.0,2000000.0,1.457500,1.457500
67,https://nayoo.co/khonkaen/posts/2244,2490000.0,1800000.0,2200000.0,1.383333,1.131818
17,https://nayoo.co/khonkaen/posts/634,10263000.0,7500000.0,9000000.0,1.368400,1.140333
13,https://nayoo.co/khonkaen/posts/597,5712000.0,4200000.0,6000000.0,1.360000,1.000000
...,...,...,...,...,...,...
51,https://nayoo.co/khonkaen/posts/1149,4000000.0,4000000.0,4500000.0,1.000000,1.000000
163,https://nayoo.co/khonkaen/posts/8682,3190000.0,3200000.0,3200000.0,1.000000,1.000000
164,https://nayoo.co/khonkaen/posts/8758,3300000.0,3500000.0,3000000.0,1.000000,1.100000
87,https://nayoo.co/khonkaen/posts/3905,1200000.0,1200000.0,1350000.0,1.000000,1.000000


## Land

In [47]:
est_2l['ประเมินโดย 1'] = est_2l['post id'].map(est_1l.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
est_2l.head(3)

<ipython-input-47-8ea92906a684>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est_2l['ประเมินโดย 1'] = est_2l['post id'].map(est_1l.set_index('post id')['ประเมินราคาซื้อ-ขาย'])


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/1033,ที่ดิน,มั่นใจ,2800000,2000000.0,"ราคาเสนอขาย 7,000-12,0000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,1800000.0
4,2,https://nayoo.co/khonkaen/posts/1711,ที่ดิน,มั่นใจ,14900000,12000000.0,"ราคาเสนอขาย 100,000-150,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,8000000.0
7,2,https://nayoo.co/khonkaen/posts/1726,ที่ดิน,มั่นใจ,90000000,70000000.0,"30,000-40,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,18000000.0


In [48]:
est_2l.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 2'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [49]:
ass_l = est_2l[['post id','ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2']]
ass_l

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2
1,https://nayoo.co/khonkaen/posts/1033,2800000,1800000.0,2000000.0
4,https://nayoo.co/khonkaen/posts/1711,14900000,8000000.0,12000000.0
7,https://nayoo.co/khonkaen/posts/1726,90000000,18000000.0,70000000.0
10,https://nayoo.co/khonkaen/posts/1732,65000000,35000000.0,60000000.0
13,https://nayoo.co/khonkaen/posts/1866,3000000,2200000.0,2500000.0
...,...,...,...,...
200,https://nayoo.co/khonkaen/posts/14539,5000000,4000000.0,4000000.0
202,https://nayoo.co/khonkaen/posts/16072,750000,500000.0,750000.0
204,https://nayoo.co/khonkaen/posts/17093,2650000,2000000.0,2500000.0
206,https://nayoo.co/khonkaen/posts/17112,1300000,1500000.0,1500000.0


เช็คและแก้ missing

In [50]:
ass_l.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2    False
dtype: bool

### MAPE

In [51]:
actual_l = ass_l[['ราคาทรัพย์']]
pred_1l = ass_l[['ประเมินโดย 1']]
pred_2l = ass_l[['ประเมินโดย 2']]

In [52]:
mpl1 = mape(actual_l,pred_1l)
mpl2 = mape(actual_l,pred_2l)
mpl12 = mape(pred_1l,pred_2l)
mpl21 = mape(pred_2l,pred_1l)

In [53]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mpl1:.2f}\n MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ {mpl2:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ {mpl12:.2f}\n MAPEราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ {mpl21:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.41
 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ 0.40
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ 0.74
 MAPEราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ 0.43


### CHECK

14 11 ลบทิ้ง

In [96]:
ass_l['Check1'] = ass_h.apply(lambda x:check_1h(x['ราคาทรัพย์'],x['ประเมินโดย 1']), axis = 1)
ass_l['Check2'] = ass_h.apply(lambda x:check_1h(x['ราคาทรัพย์'],x['ประเมินโดย 2']), axis = 1)
ass_l

<ipython-input-96-05f0818257d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_l['Check1'] = ass_h.apply(lambda x:check_1h(x['ราคาทรัพย์'],x['ประเมินโดย 1']), axis = 1)
<ipython-input-96-05f0818257d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ass_l['Check2'] = ass_h.apply(lambda x:check_1h(x['ราคาทรัพย์'],x['ประเมินโดย 2']), axis = 1)


,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
1,https://nayoo.co/khonkaen/posts/1033,2800000,1800000.0,2000000.0,1.080000,1.000000
4,https://nayoo.co/khonkaen/posts/1711,14900000,8000000.0,12000000.0,NaN,NaN
7,https://nayoo.co/khonkaen/posts/1726,90000000,18000000.0,70000000.0,1.257143,1.189189
10,https://nayoo.co/khonkaen/posts/1732,65000000,35000000.0,60000000.0,NaN,NaN
13,https://nayoo.co/khonkaen/posts/1866,3000000,2200000.0,2500000.0,1.360000,1.000000
...,...,...,...,...,...,...
200,https://nayoo.co/khonkaen/posts/14539,5000000,4000000.0,4000000.0,NaN,NaN
202,https://nayoo.co/khonkaen/posts/16072,750000,500000.0,750000.0,NaN,NaN
204,https://nayoo.co/khonkaen/posts/17093,2650000,2000000.0,2500000.0,NaN,NaN
206,https://nayoo.co/khonkaen/posts/17112,1300000,1500000.0,1500000.0,NaN,NaN


In [97]:
ass_l.sort_values(by=['Check1'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
186,https://nayoo.co/khonkaen/posts/12139,30000000,28000000.0,29000000.0,1.521346,1.069054
13,https://nayoo.co/khonkaen/posts/1866,3000000,2200000.0,2500000.0,1.360000,1.000000
39,https://nayoo.co/khonkaen/posts/3515,3300000,3150000.0,3500000.0,1.257143,1.000000
7,https://nayoo.co/khonkaen/posts/1726,90000000,18000000.0,70000000.0,1.257143,1.189189
166,https://nayoo.co/khonkaen/posts/8226,8000000,6500000.0,7500000.0,1.234043,1.000000
...,...,...,...,...,...,...
200,https://nayoo.co/khonkaen/posts/14539,5000000,4000000.0,4000000.0,NaN,NaN
202,https://nayoo.co/khonkaen/posts/16072,750000,500000.0,750000.0,NaN,NaN
204,https://nayoo.co/khonkaen/posts/17093,2650000,2000000.0,2500000.0,NaN,NaN
206,https://nayoo.co/khonkaen/posts/17112,1300000,1500000.0,1500000.0,NaN,NaN


In [98]:
ass_l.sort_values(by=['Check2'], ascending=False)

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2,Check1,Check2
7,https://nayoo.co/khonkaen/posts/1726,90000000,18000000.0,70000000.0,1.257143,1.189189
164,https://nayoo.co/khonkaen/posts/7838,46472000,35000000.0,40000000.0,1.000000,1.100000
45,https://nayoo.co/khonkaen/posts/7044,1750000,1500000.0,1800000.0,1.088889,1.088889
168,https://nayoo.co/khonkaen/posts/8326,2800000,2200000.0,3000000.0,1.085714,1.085714
33,https://nayoo.co/khonkaen/posts/2206,6000000,5000000.0,5500000.0,1.115714,1.084722
...,...,...,...,...,...,...
200,https://nayoo.co/khonkaen/posts/14539,5000000,4000000.0,4000000.0,NaN,NaN
202,https://nayoo.co/khonkaen/posts/16072,750000,500000.0,750000.0,NaN,NaN
204,https://nayoo.co/khonkaen/posts/17093,2650000,2000000.0,2500000.0,NaN,NaN
206,https://nayoo.co/khonkaen/posts/17112,1300000,1500000.0,1500000.0,NaN,NaN


## Condo

In [54]:
condo.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 1'}, inplace=True)

In [55]:
ass_c = condo[['post id','ราคาทรัพย์','ประเมินโดย 1']]
ass_c

,post id,ราคาทรัพย์,ประเมินโดย 1
0,32.0,3500000.0,3500000.0
1,36.0,1930000.0,2200000.0
2,603.0,2438000.0,2800000.0
3,1395.0,1600000.0,2300000.0
4,1443.0,1860000.0,1800000.0
5,1609.0,1490000.0,1500000.0
6,1640.0,1280000.0,1600000.0
7,1641.0,4290000.0,2700000.0
8,1650.0,1700000.0,1550000.0
9,1840.0,3790000.0,2800000.0


เช็คและแก้ missing

In [56]:
ass_c.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
dtype: bool

### MAPE

In [57]:
actual_c = ass_c[['ราคาทรัพย์']]
pred_1c = ass_c[['ประเมินโดย 1']]

In [58]:
mpc1 = mape(actual_c,pred_1c)

In [59]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mpc1:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.17


## Town homw

In [60]:
est_2t['ประเมินโดย 1'] = est_2t['post id'].map(est_1t.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
est_2t.head(3)

<ipython-input-60-309d826ee320>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est_2t['ประเมินโดย 1'] = est_2t['post id'].map(est_1t.set_index('post id')['ประเมินราคาซื้อ-ขาย'])


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/548,ทาวน์โฮม,มั่นใจ,1045000.0,1500000.0,ราคาซื้อขาย 1-2 ล้าน/คูหา,NaN,1 ขายไม่ได้แน่นอน,NaN,NaN,NaN,NaN,700000.0
3,2,https://nayoo.co/khonkaen/posts/549,ทาวน์โฮม,มั่นใจ,935000.0,1500000.0,ราคาเสนอขาย 1-2 ล้าน/คูหา,NaN,1 ขายไม่ได้แน่นอน,NaN,NaN,NaN,NaN,700000.0
5,2,https://nayoo.co/khonkaen/posts/579,ทาวน์โฮม,มั่นใจ,3520000.0,4000000.0,ราคาเสนอขาย 3-5 ล้านบาท/คูหา,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,NaN,NaN,NaN,NaN,3500000.0


In [61]:
est_2t.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 2'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [62]:
ass_th = est_2t[['post id','ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2']]
ass_th

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2
1,https://nayoo.co/khonkaen/posts/548,1045000.0,700000.0,1500000.0
3,https://nayoo.co/khonkaen/posts/549,935000.0,700000.0,1500000.0
5,https://nayoo.co/khonkaen/posts/579,3520000.0,3500000.0,4000000.0
7,https://nayoo.co/khonkaen/posts/580,3850000.0,3500000.0,3200000.0
9,https://nayoo.co/khonkaen/posts/582,990000.0,1000000.0,800000.0
...,...,...,...,...
235,https://nayoo.co/khonkaen/posts/18547,1430000.0,1000000.0,1200000.0
236,https://nayoo.co/khonkaen/posts/18551,1980000.0,1500000.0,1700000.0
237,https://nayoo.co/khonkaen/posts/19434,2000000.0,1600000.0,1900000.0
238,https://nayoo.co/khonkaen/posts/19501,1790000.0,1600000.0,1700000.0


เช็คและแก้ missing

In [63]:
ass_th.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2    False
dtype: bool

### MAPE

In [64]:
actual_th = ass_th[['ราคาทรัพย์']]
pred_1th = ass_th[['ประเมินโดย 1']]
pred_2th = ass_th[['ประเมินโดย 2']]

In [65]:
mpth1 = mape(actual_th,pred_1th)
mpth2 = mape(actual_th,pred_2th)
mpth12 = mape(pred_1th,pred_2th)
mpth21 = mape(pred_2th,pred_1th)

In [66]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mpth1:.2f}\n MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ {mpth2:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ {mpth12:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ {mpth21:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.11
 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ 0.09
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ 0.11
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 1 คือ 0.11


## Shop house

In [67]:
est_2s['ประเมินโดย 1'] = est_2s['post id'].map(est_1s.set_index('post id')['ประเมินราคาซื้อ-ขาย'])
est_2s.head(3)

<ipython-input-67-f42338c5bcdb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est_2s['ประเมินโดย 1'] = est_2s['post id'].map(est_1s.set_index('post id')['ประเมินราคาซื้อ-ขาย'])


,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,ประเมินโดย 1
1,2,https://nayoo.co/khonkaen/posts/6,อาคารพาณิชย์/สำนักงาน,มั่นใจ,7000000.0,8000000.0,"ราคาเสนอขาย 4,000,000-6,000,000 บาท/คูหา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0,ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,NaN,NaN,NaN,6000000.0
3,2,https://nayoo.co/khonkaen/posts/17,อาคารพาณิชย์/สำนักงาน,มั่นใจ,12000000.0,12000000.0,ราคาเสนอขาย 4 - 5 ล้านบาท/คูหา,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2.0,ขายได้ ใช้ระยะเวลา 4-6 เดือน,NaN,NaN,NaN,12000000.0
5,2,https://nayoo.co/khonkaen/posts/21,อาคารพาณิชย์/สำนักงาน,มั่นใจ,22500000.0,20000000.0,ราคาเสนอขาย 6-8 ล้านบาท / คูหา,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3.0,ขายได้ ใช้ระยะเวลา 7-12 เดือน,NaN,NaN,NaN,13500000.0


In [68]:
est_2s.rename(columns={'ประเมินราคาซื้อ-ขาย' :'ประเมินโดย 2'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [69]:
ass_sh = est_2s[['post id','ราคาทรัพย์','ประเมินโดย 1','ประเมินโดย 2']]
ass_sh

,post id,ราคาทรัพย์,ประเมินโดย 1,ประเมินโดย 2
1,https://nayoo.co/khonkaen/posts/6,7000000.0,6000000.0,8000000.0
3,https://nayoo.co/khonkaen/posts/17,12000000.0,12000000.0,12000000.0
5,https://nayoo.co/khonkaen/posts/21,22500000.0,13500000.0,20000000.0
7,https://nayoo.co/khonkaen/posts/22,3500000.0,4000000.0,4000000.0
9,https://nayoo.co/khonkaen/posts/24,3900000.0,2200000.0,3000000.0
11,https://nayoo.co/khonkaen/posts/30,4800000.0,3600000.0,4000000.0
13,https://nayoo.co/khonkaen/posts/35,2400000.0,2500000.0,2100000.0
15,https://nayoo.co/khonkaen/posts/497,5000000.0,3800000.0,5000000.0
17,https://nayoo.co/khonkaen/posts/498,7000000.0,7000000.0,8000000.0
19,https://nayoo.co/khonkaen/posts/499,9000000.0,8000000.0,10000000.0


เช็คและแก้ missing

In [70]:
ass_sh.isnull().any()

post id         False
ราคาทรัพย์      False
ประเมินโดย 1    False
ประเมินโดย 2    False
dtype: bool

### MAPE

In [71]:
actual_sh = ass_sh[['ราคาทรัพย์']]
pred_1sh = ass_sh[['ประเมินโดย 1']]
pred_2sh = ass_sh[['ประเมินโดย 2']]

In [72]:
mpsh1 = mape(actual_sh,pred_1sh)
mpsh2 = mape(actual_sh,pred_2sh)
mpsh12 = mape(pred_1sh,pred_2sh)
mpsh21 = mape(pred_2sh,pred_1sh)

In [73]:
print(f' MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ {mpsh1:.2f}\n MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ {mpsh2:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ {mpsh12:.2f}\n MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 2 คือ {mpsh21:.2f}')

 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 1 คือ 0.18
 MAPE ราคาทรัพย์กับผู้ประเมินคนที่ 2 คือ 0.12
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 1 กับ ผู้ประเมินคนที่ 2 คือ 0.22
 MAPE ราคาตั้งขายของผู้ประเมินคนที่ 2 กับ ผู้ประเมินคนที่ 2 คือ 0.17
